In [3]:
!pip install -U pip
!pip install llvmlite==0.31.0

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0
    Uninstalling pip-23.0:
      Successfully uninstalled pip-23.0
Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install numba==0.48

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip show numba
!pip show llvmlite

Name: numba
Version: 0.48.0
Summary: compiling Python code using LLVM
Home-page: http://numba.github.com
Author: Anaconda, Inc.
Author-email: numba-users@continuum.io
License: BSD
Location: /home/ilia/.local/lib/python3.8/site-packages
Requires: llvmlite, numpy, setuptools
Required-by: efficient-latent, librosa, resampy
Name: llvmlite
Version: 0.31.0
Summary: lightweight wrapper around basic LLVM functionality
Home-page: http://llvmlite.pydata.org
Author: Continuum Analytics, Inc.
Author-email: numba-users@continuum.io
License: BSD
Location: /home/ilia/.local/lib/python3.8/site-packages
Requires: 
Required-by: numba


In [6]:
%cd ~/projects/audio-captioning
!rm -rf AudioCaption
!git clone https://github.com/wsntxxn/AudioCaption
%cd AudioCaption
!python3 -m pip install -r requirements.txt

/home/ilia/projects/audio-captioning
Cloning into 'AudioCaption'...
remote: Enumerating objects: 794, done.
remote: Counting objects: 100% (794/794), done.
remote: Compressing objects: 100% (425/425), done.
remote: Total 794 (delta 434), reused 702 (delta 342), pack-reused 0
Receiving objects: 100% (794/794), 10.07 MiB | 9.03 MiB/s, done.
Resolving deltas: 100% (434/434), done.
/home/ilia/projects/audio-captioning/AudioCaption


In [7]:
!pip install -e .

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///home/ilia/projects/audio-captioning/AudioCaption
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/richermans/HEAR2021_EfficientLatent.git to /tmp/pip-install-znjmuk0q/efficient-latent_cc95ba79f87f496da2cb29e967d76a62
  Running command git clone --filter=blob:none --quiet https://github.com/richermans/HEAR2021_EfficientLatent.git /tmp/pip-install-znjmuk0q/efficient-latent_cc95ba79f87f496da2cb29e967d76a62
  Resolved https://github.com/richermans/HEAR2021_EfficientLatent.git to commit 49d5a5c1992ce3ebd5efebfcb792dd9f9976170d
  Preparing metadata (setup.py) ... done
  Running setup.py develop for audiocaptioning


In [23]:
%cd ~/projects/audio-captioning/AudioCaption
!rm -rf audiofiles
!tar -zxf ../data.tar.gz
!mkdir audiofiles
!mv *wav audiofiles

/home/ilia/projects/audio-captioning/AudioCaption
['bip.wav', 'dog.wav', 'bell.wav', 'gong.wav', 'atm01.wav', 'beebee.wav', 'drawer.wav', 'STE-008.wav', 'balloon.wav', 'buzzing.wav', 'cough_1.wav', 'cough_2.wav', 'flute_1.wav', 'barbecue.wav', 'brushing.wav', 'fans_hbf.wav', 'Gouttiere.wav', 'crackling.wav', 'air duster.wav', 'babyrattle.wav', 'cat eating.wav', 'chainsaw-1.wav', 'chainsaw-2.wav', 'clickclick.wav', 'dishwasher.wav', 'CDmaschine1.wav', 'Plastic bag.wav', 'cooker_hood.wav', 'drilling_01.wav', 'gobbler_bod.wav', 'Forest Track.wav', 'Toilet_Flush.wav', 'carrotpeeler.wav', 'coffee-maker.wav', 'creaky_chair.wav', 'cuttingpaper.wav', 'distributeur.wav', 'Saint-Gervais.wav', 'creaking_wood.wav', 'cutlerydrawer.wav', 'finger-clicks.wav', 'Forest Track 2.wav', 'Police sirens2.wav', 'Purring Cat 01.wav', 'bagofpistachios.wav', 'deodorant_spray.wav', 'electric-beater.wav', 'Ijmuiden harbour.wav', 'air duster short.wav', 'cracking_peanuts.wav', 'crunching cherry.wav', 'cutlery on ta

In [9]:
!unzip ~/projects/audio-captioning/experiments.zip -d ~/projects/audio-captioning/AudioCaption/

Archive:  /home/ilia/projects/audio-captioning/experiments.zip
  inflating: /home/ilia/projects/audio-captioning/AudioCaption/experiments/clotho_v2/train_val/TransformerModel/cnn14rnn_trm/seed_1/train.log  
  inflating: /home/ilia/projects/audio-captioning/AudioCaption/experiments/clotho_v2/train_val/TransformerModel/cnn14rnn_trm/seed_1/config.yaml  
  inflating: /home/ilia/projects/audio-captioning/AudioCaption/experiments/clotho_v2/train_val/TransformerModel/cnn14rnn_trm/seed_1/best.pth  
  inflating: /home/ilia/projects/audio-captioning/AudioCaption/experiments/clotho_v2/train_val/TransformerModel/cnn14rnn_trm/seed_1/last.pth  
  inflating: /home/ilia/projects/audio-captioning/AudioCaption/experiments/clotho_v2/train_val/TransformerModel/cnn14rnn_trm/seed_1/swa.pth  
  inflating: /home/ilia/projects/audio-captioning/AudioCaption/experiments/clotho_v2/train_val/TransformerModel/cnn14rnn_trm/seed_1/run/events.out.tfevents.1678448323.interns-0.5697.0  


In [15]:
!/home/ilia/miniconda3/envs/py38/bin/python --version

Python 3.8.16


In [26]:
import sys
sys.path.append("~/projects/audio-captioning/AudioCaption/captioning")
sys.path.append("~/projects/audio-captioning/AudioCaption")
sys.path.append("~/projects/audio-captioning")
audiofiles = !ls -1 audiofiles
print(sorted(audiofiles, key=lambda s: len(s)))
%cd ~/projects/audio-captioning/AudioCaption
for filename_ix, filename in enumerate(audiofiles):
    print(f"Trying to infer on {filename}")
    !/home/ilia/miniconda3/envs/py38/bin/python captioning/pytorch_runners/inference_waveform.py \
             'audiofiles/{filename}' \
             {filename_ix}.json \
             --checkpoint=experiments/clotho_v2/train_val/TransformerModel/cnn14rnn_trm/seed_1/swa.pth
    break



/home/ilia/projects/audio-captioning/AudioCaption
Trying to infer on 08-01-01-Silvester in Berlin-fireworks-1.wav
  0%|                                                     | 0/1 [00:00<?, ?it/s]rnn.network.weight_hh_l0', 'encoder.rnn.network.bias_ih_l0', 'encoder.rnn.network.bias_hh_l0', 'encoder.rnn.network.weight_ih_l0_reverse', 'encoder.rnn.network.weight_hh_l0_reverse', 'encoder.rnn.network.bias_ih_l0_reverse', 'encoder.rnn.network.bias_hh_l0_reverse', 'encoder.rnn.network.weight_ih_l1', 'encoder.rnn.network.weight_hh_l1', 'encoder.rnn.network.bias_ih_l1', 'encoder.rnn.network.bias_hh_l1', 'encoder.rnn.network.weight_ih_l1_reverse', 'encoder.rnn.network.weight_hh_l1_reverse', 'encoder.rnn.network.bias_ih_l1_reverse', 'encoder.rnn.network.bias_hh_l1_reverse', 'encoder.rnn.network.weight_ih_l2', 'encoder.rnn.network.weight_hh_l2', 'encoder.rnn.network.bias_ih_l2', 'encoder.rnn.network.bias_hh_l2', 'encoder.rnn.network.weight_ih_l2_reverse', 'encoder.rnn.network.weight_hh_l2_reverse',

In [ ]:
import json
results_fnames = !ls -1 *json
results_data = []
for res_fname in results_fnames:
    potential_info = json.load(open(res_fname, 'r'))
    if potential_info:
        for k, v in potential_info.items():
            if k.endswith('wav'):
                results_data.append({"fname": k, "caption":v})
import pandas as pd
pd.DataFrame(results_data).to_csv("v2023_analyses.csv")